In [ ]:
import polars as pl
import numpy as np

import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from config import CONFIG
from data_manager import DataManager
from nn_module import PowerForecastDataset, TSCNNPowerForecast
from nn_trainer import TSCNNTrainer
from utils import same_seed

In [ ]:
same_seed(2025)

In [ ]:
manager = DataManager()

In [ ]:
train_data, valid_data = manager.load_weather_data(mode="train", split=True)
train_data = manager.apply_feature_engineering(train_data)
valid_data = manager.apply_feature_engineering(valid_data)
train_data = manager.standardize(train_data, "global", "train")
valid_data = manager.standardize(valid_data, "global", "valid")

In [ ]:
test_data = manager.load_weather_data(mode="test")
test_data = manager.apply_feature_engineering(test_data)
test_data = manager.standardize(test_data, "global", "test")

In [ ]:
train_target_data, valid_target_data = manager.load_target_data(mode="train", split=True)
test_target_data = manager.load_target_data(mode="test")

In [ ]:
NN_FEATURES = CONFIG.FEATURES + ["lat_enc", "lon_enc", "plant_id"]
NUM_FEATURES = len(NN_FEATURES)

In [ ]:
def input_pl2numpy(df: pl.DataFrame) -> np.ndarray:
    
    return (
        df
        .filter(pl.col("source") == "NWP_21")
        .sort(["plant_id", "forecast_time", "lat", "lon"])
        .select(NN_FEATURES)
        .to_numpy()
        .squeeze()
        .reshape(-1, 24, 11, 11, NUM_FEATURES))
    
train_data_pl = pl.concat([train_df for train_df in train_data.values()])
valid_data_pl = pl.concat([valid_df for valid_df in valid_data.values()])
test_data_pl = pl.concat([test_df for test_df in test_data.values()])

train_data_numpy = input_pl2numpy(train_data_pl)
valid_data_numpy = input_pl2numpy(valid_data_pl)
test_data_numpy = input_pl2numpy(test_data_pl)
train_data_numpy.shape, valid_data_numpy.shape, test_data_numpy.shape

In [ ]:
def target_pl2numpy(df: pl.DataFrame) -> np.ndarray:
    return (
        df
        .sort(["plant_id_target", "forecast_time"])
        .select("target")
        .to_numpy()
        .squeeze()
        .reshape(-1, 96))

train_target_pl = pl.concat([train_target_df for train_target_df in train_target_data.values()])
valid_target_pl =  pl.concat([valid_target_df for valid_target_df in valid_target_data.values()])
test_target_pl = pl.concat([test_target_df for test_target_df in test_target_data.values()])

train_target_numpy = target_pl2numpy(train_target_pl)
valid_target_numpy = target_pl2numpy(valid_target_pl)
test_target_numpy = target_pl2numpy(test_target_pl)
    
train_target_numpy.shape, valid_target_numpy.shape, test_target_numpy.shape

In [ ]:
BATCH_SIZE = 1
train_dataset = PowerForecastDataset(train_data_numpy, train_target_numpy)
valid_dataset = PowerForecastDataset(valid_data_numpy, valid_target_numpy)
test_dataset = PowerForecastDataset(test_data_numpy, test_target_numpy)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

in_channels = NUM_FEATURES
hidden_dim = 128
num_layers = 2
dropout = 0.2
output_steps = 96
model = TSCNNPowerForecast(in_channels, hidden_dim, num_layers, dropout, output_steps)

LR = 1e-4
WEIGRT_DECAY = 0.0001
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(), 
    lr=LR, 
    weight_decay=WEIGRT_DECAY
)

EPOCHS = 10
trainer = TSCNNTrainer(model, criterion, optimizer, EPOCHS)
trainer.fit(train_dataloader, valid_dataloader)

In [ ]:
test_pred = np.concatenate(trainer.inference(test_dataloader)).reshape(-1, 1).squeeze()
pred_df = (
    test_target_pl
    .with_columns([
        pl.Series(test_pred).alias("predict")
    ])
    .rename({
        "forecast_time": "Time",
        "predict": "Power",
    })
    .select("Time", "Power", "plant_id_target")
)

# 保存提交文件
for (plant_id, ), df in pred_df.group_by("plant_id_target"):
    df = (
        df.with_columns([
            pl.col("Time").dt.strftime("%Y/%-m/%-d %H:%M").alias("Time")
        ])
        .drop("plant_id_target")
        .to_pandas()
        .set_index("Time")
    )
    df.to_csv(f"{CONFIG.OUTPUT_PATH}/output{plant_id}.csv")